In [ ]:
%load_ext autoreload

In [ ]:
import numpy as np
import os
import mcubes
import meshcat
import pydrake
from pydrake.geometry import SceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.multibody.parsing import Parser, LoadModelDirectives, ProcessModelDirectives
from pydrake.multibody.tree import RevoluteJoint
from pydrake.all import ConnectMeshcatVisualizer, InverseKinematics, RigidTransform, RotationMatrix
from pydrake.all import BsplineTrajectoryThroughUnionOfHPolyhedra, IrisInConfigurationSpace, IrisOptions, Rgba
import time
from meshcat import Visualizer
from functools import partial

import ipywidgets as widgets
from IPython.display import display

#from meshcat_cpp_utils import StartMeshcat, MeshcatJointSliders
from pydrake.all import MeshcatVisualizerCpp, MeshcatVisualizerParams, Role
import rrt, utils, prm# spp

In [ ]:
# Setup meshcat
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])

In [ ]:
vis = Visualizer(zmq_url=zmq_url)
vis.delete()

#model_file = FindResourceOrThrow("drake/manipulation/models/iiwa_description/urdf/iiwa14_polytope_collision.urdf")
model_file = FindResourceOrThrow("drake/manipulation/models/iiwa_description/iiwa7/iiwa7_with_box_collision.sdf")
model_file_2 = FindResourceOrThrow("drake/manipulation/models/iiwa_description/iiwa7_2/iiwa7_with_box_collision.sdf")
#model_file = FindResourceOrThrow("drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf")
#model_file = FindResourceOrThrow("drake/manipulation/models/iiwa_description/sdf/iiwa14_no_collision.sdf")

#box_file = FindResourceOrThrow("drake/sandbox/assets/box.urdf")
box_file_1 = FindResourceOrThrow("drake/sandbox/assets/shelves.sdf")
box_file_2 = FindResourceOrThrow("drake/sandbox/assets/shelves2.sdf")

models =[]

builder = DiagramBuilder()

plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
models.append(Parser(plant, scene_graph).AddModelFromFile(model_file))
models.append(Parser(plant, scene_graph).AddModelFromFile(model_file_2))

models.append(Parser(plant, scene_graph).AddModelFromFile(box_file_1))
models.append(Parser(plant, scene_graph).AddModelFromFile(box_file_2))

# locs = [ [0,.2,0], [0,-.2,0], [.6, 0, .4],[-.6, 0, .4]] 
locs = [ [0,.2,0], [0,-.2,0], [.6, .3, .4],[.6, -.3, .4]] 
idx = 0
for model in models:
    plant.WeldFrames(plant.world_frame(), plant.get_body(plant.GetBodyIndices(model)[0]).body_frame(), RigidTransform(locs[idx]))
    idx +=1
# plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("base", model), RigidTransform(locs[idx]))

plant.Finalize()

visualizer = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, delete_prefix_on_load=False, )

diagram = builder.Build()
visualizer.load()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
diagram.Publish(context)



In [ ]:
sliders = []
for joint_idx in range(1,8):
    for iiwa_idx in [0,1]:
        sliders.append(
            widgets.FloatSlider(
            min=plant.GetJointByName(f"iiwa_joint_{joint_idx}",models[iiwa_idx]).position_lower_limit(), 
            max=plant.GetJointByName(f"iiwa_joint_{joint_idx}",models[iiwa_idx]).position_upper_limit(), 
            value=0, 
            description=f'model{iiwa_idx}_iiwa_joint_{joint_idx}'
            )
        )

ik = InverseKinematics(plant, plant_context)
collision_constraint = ik.AddMinimumDistanceConstraint(0.001, 0.001)

def eval_cons(q, c, tol):
    return 1-1*float(c.evaluator().CheckSatisfied(q, tol))
    
col_func_handle = partial(eval_cons, c=collision_constraint, tol=0.01)
    
def showres(q):
    col = col_func_handle(q)
    if col:
        vis["collision"].set_object(
                meshcat.geometry.Sphere(0.1), meshcat.geometry.MeshLambertMaterial(color=0xFF0000))
        vis["collision"].set_transform(
                meshcat.transformations.translation_matrix([0, 1.0, 1.0]))
    else:
        vis["collision"].set_object(
                meshcat.geometry.Sphere(0.1), meshcat.geometry.MeshLambertMaterial(color=0x00FF00))
        vis["collision"].set_transform(
                meshcat.transformations.translation_matrix([0, 1.0, 1.0]))
    diagram.Publish(context)
    print("              ", end = "\r")
    print(col , end = "\r")

    
def set_joint_angles(vals):
    for joint_idx in range(1,8):
        for iiwa_idx in [0,1]:
            plant.GetJointByName(f"iiwa_joint_{joint_idx}",models[iiwa_idx]).set_angle(plant_context, vals[joint_idx-1])
    
    
    
q_init = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
q = q_init.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    #print(q, end="\r")
    showres(q)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1


In [ ]:
for slider in sliders:
    display(slider)
display(vis.jupyter_cell())

In [ ]:
#problem config
start = np.array([-0.87, 0.83, 1.21, 0.21, 0., 0., -1.59, -1.49, 0., 0., -1.19, 0., 0., 0.])
target = np.array([-0.83, 0.87, 0.21, 1.21, 0., 0., -1.49, -1.59, 0., 0., 0., -1.19, 0., 0.])
showres(start)
limits =[plant.GetPositionLowerLimits(), plant.GetPositionUpperLimits()]

In [ ]:
showres(target)

In [ ]:
showres(np.zeros(14))

In [ ]:
PRM = prm.PRM( 
            limits,
            num_points = 100000,
            col_func_handle = col_func_handle,
            num_neighbours = 20, 
            dist_thresh = 5.0,
            num_col_checks = 10,
            verbose = True,
            plotcallback = None
            )

PRM.add_start_end(start, target)
prm_path, _ = PRM.find_shortest_path()
print(f'PRM path length: {len(prm_path)}')

In [ ]:
prm_traj= utils.PWLinTraj(prm_path, 5.0)
substeps = 1000
utils.animate(prm_traj, showres, substeps, 1*substeps)
utils.plot(prm_traj, substeps, 1*substeps)

In [ ]:
print(f'PRM path shape: {np.array(prm_path).shape}')
subsampled_prm_path = prm_path[1:-1]
print(f'PRM path shape subsampled for IRIS seed points: {np.array(subsampled_prm_path).shape}')

In [ ]:
# DOESN'T CONVERGE YET

RRT = rrt.RRT(start = target,
              goal = start,
              limits = limits,
              col_func_handle=col_func_handle,
              max_extend_length=.001,
              extend_steps=0.0005,
              init_goal_sample_rate=0.00,
              goal_sample_rate_scaler=0.35,
              verbose = True, 
              plotcallback = None)

_, path = RRT.run(30000)
print(f'RRT path length: {len(rrt_path)}')

In [ ]:
rrt_traj= utils.PWLinTraj(rrt_path, 5.0)
substeps = 1000
utils.animate(rrt_traj, showres, substeps, 1*substeps)
utils.plot(rrt_traj, substeps, 1*substeps)

In [ ]:
print(f'RRT path shape: {np.array(rrt_path).shape}')
subsampled_rrt_path = rrt_path[1:-1:10]
print(f'RRT path shape subsampled for IRIS seed points: {np.array(subsampled_rrt_path).shape}')

In [ ]:
def do_iris(q_seed, iris_options):
    start_time = time.time()
    hpoly = IrisInConfigurationSpace(plant, plant_context, q_seed, iris_options)
    ellipse = hpoly.MaximumVolumeInscribedEllipsoid()
    print("Time: %6.2f \tVolume: %6.2f \tCenter:" % (time.time() - start_time, ellipse.Volume()),
          ellipse.center(), flush=True)
    return ellipse, hpoly

iris_options = IrisOptions()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 50
iris_options.enable_ibex = False

regions = []
ellipses = []
its = 40
# seed_points = rrt_path
# seed_points = [start] + subsampled_rrt_path + [target]
seed_points = [start] + subsampled_prm_path + [target]
# seed_points = [start, target]

it = 0
for point in seed_points:
    ell, reg = do_iris(point, iris_options)
    
    if ell.Volume() < 1000.0:
        regions.append(reg)
        ellipses.append(ell)
        
        set_joint_angles(ell.center())
        tf = plant.EvalBodyPoseInWorld(plant_context, plant.get_body(pydrake.multibody.tree.BodyIndex(6)))
        mat = meshcat.geometry.MeshLambertMaterial(color=0x0029F1)
        mat.reflectivity = 1.0
        vis['centers']['points' + str(it)].set_object(
                    meshcat.geometry.Sphere(0.02), mat)
        vis['centers']['points' + str(it)].set_transform(tf.GetAsMatrix4())
    it += 1

In [ ]:
start_time = time.time()
spp = BsplineTrajectoryThroughUnionOfHPolyhedra(start, target, regions)
spp.set_max_velocity(.8 * np.ones(7))
spp.set_extra_control_points_per_region(2)

print(spp.num_regions())
traj = spp.Solve()
print(time.time() - start_time)
print(traj)

print(traj.start_time())
print(traj.end_time())
#visualize
maxit = 60
pts = []
for it in range(maxit):
    print(f'Iteration {it}/{maxit-1}')
    pts.append(traj.value(it*traj.end_time()/maxit))
    mat = meshcat.geometry.MeshLambertMaterial(color=0xFFF812)
    mat.reflectivity = 1.0
    set_joint_angles(pts[-1].reshape(-1,))
    tf= plant.EvalBodyPoseInWorld(plant_context, plant.get_body(pydrake.multibody.tree.BodyIndex(3)))
   
     
    mat = meshcat.geometry.MeshLambertMaterial(color=0x0029F1)
    mat.reflectivity = 1.0
    vis['traj']['points' + str(it)].set_object(
                meshcat.geometry.Sphere(0.02), mat)
    vis['traj']['points' + str(it)].set_transform(
                meshcat.transformations.translation_matrix(tf.translation()))
    

In [ ]:
pts = []
for it in range(100):
    pts.append(traj.value(it*traj.end_time()/maxit))
    mat = meshcat.geometry.MeshLambertMaterial(color=0xFFF812)
    mat.reflectivity = 1.0
    set_joint_angles(pts[-1].reshape(-1,))
    tf= plant.EvalBodyPoseInWorld(plant_context, plant.get_body(pydrake.multibody.tree.BodyIndex(3)))
   
     
    mat = meshcat.geometry.MeshLambertMaterial(color=0x0029F1)
    mat.reflectivity = 1.0
    vis['traj']['points' + str(it)].set_object(
                meshcat.geometry.Sphere(0.02), mat)
    vis['traj']['points' + str(it)].set_transform(
                meshcat.transformations.translation_matrix(tf.tranlation()))

In [ ]:
# traj
substeps = 1000
utils.animate(traj, showres, substeps, 1*substeps)
utils.plot(traj, substeps, 1*substeps)

In [ ]:
# Manual reversal of IRIS trajectory

import matplotlib.pyplot as plt

#loop
idx = 0
going_fwd = True
time_points = np.linspace(0, traj.end_time(), substeps) 

for _ in range(substeps):
    #print(idx)
    q = traj.value(time_points[substeps-idx-1])
    showres(q.reshape(-1,))
    if going_fwd:
        if idx + 1 < substeps:
            idx += 1
        else:
            going_fwd = False
            idx -=1
    else:
        if idx-1 >= 0:
            idx -=1
        else:
            going_fwd = True
            idx +=1


#loop
idx = 0
going_fwd = True
time_points = np.linspace(0, traj.end_time(), substeps) 
traj_list = []

for _ in range(substeps):
    #print(idx)
    traj_list.append(traj.value(time_points[substeps-idx-1]).reshape(-1,))
    if going_fwd:
        if idx + 1 < substeps:
            idx += 1
        else:
            going_fwd = False
            idx -=1
    else:
        if idx-1 >= 0:
            idx -=1
        else:
            going_fwd = True
            idx +=1

traj_arr = np.array(traj_list)

fig, ax = plt.subplots(1,1, figsize=(10,6), dpi=72*3)
data_dims = traj_arr.shape[1]
for joint_idx in range(data_dims):
    ax.plot(np.arange(len(traj_arr[:,joint_idx])),traj_arr[:,joint_idx], label=f'Joint {joint_idx+1}')
ax.legend(loc='upper center', ncol=data_dims)
ax.set_ylim([-np.pi, np.pi])
plt.show()

In [ ]:
traj= utils.PWLinTraj(path, 5.0)
substeps = 1000
utils.animate(traj, showres, substeps, 1*substeps)

In [ ]:
import spp


In [ ]:
from pydrake.geometry.optimization import 
